In [4]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from pathlib import Path
from scipy.stats import ttest_rel

In [5]:
results_file = Path(r"../DVlog/trained_models/test_metrics.csv")

# load in the file with the results on the test set
df_results = pd.read_csv(results_file, sep=";")
df_results.head()

,Unnamed: 0,name,seed,precision,recall,f1,f1_m,f1_f,Eq accuracy,eq opportunity,pred equality,unpriv_TPR,unpriv_FPR,priv_TPR,priv_FPR
0,0,unimodal_bias_mf-group_mpnet_sent,0,0.929159,0.927273,0.926964,0.877117,0.953358,0.920030,0.900000,1.408046,0.900000,0.172414,1.000000,0.122449
1,1,unimodal_bias_mf-group_mpnet_sent,1,0.927386,0.927273,0.927192,0.877193,0.953522,0.919951,0.881111,1.351724,0.866667,0.137931,0.983607,0.102041
2,2,unimodal_bias_mf-group_mpnet_sent,1123,0.918511,0.915152,0.914619,0.859389,0.943915,0.910452,0.900000,1.448276,0.900000,0.206897,1.000000,0.142857
3,3,unimodal_bias_mf-group_mpnet_sent,3407,0.933333,0.933333,0.933299,0.894672,0.953522,0.938282,0.881111,1.013793,0.866667,0.103448,0.983607,0.102041
4,4,unimodal_bias_mf-group_mpnet_sent,42,0.934632,0.933333,0.933110,0.876737,0.962873,0.910543,0.948889,2.534483,0.933333,0.206897,0.983607,0.081633


In [6]:
eqodds_file = Path(r"../DVlog/trained_models/eqodds_metrics.csv")

# load in the file with the eq odds results on the test set
df_eqodds = pd.read_csv(eqodds_file, sep=";")

# setup the dataframe
df_eqodds = df_eqodds[df_eqodds["dataset"] == "test"]
df_eqodds.drop("dataset", axis=1, inplace=True)
df_eqodds.head()

,Unnamed: 0,name,seed,precision,recall,f1,f1_m,f1_f,eq opportunity,Eq accuracy,pred equality
2,2,eqodds,0,0.934,0.933,0.933,0.877,0.963,0.86,0.91,1.69
5,5,eqodds,1,0.928,0.927,0.927,0.894,0.944,0.86,0.95,0.68
8,8,eqodds,42,0.906,0.903,0.902,0.842,0.935,0.92,0.90,1.69
11,11,eqodds,1123,0.934,0.933,0.933,0.895,0.954,0.91,0.94,1.69
14,14,eqodds,3407,0.933,0.933,0.933,0.895,0.954,0.93,0.94,1.69


In [7]:
df = pd.concat([df_results, df_eqodds], ignore_index=True)
df.head()

,Unnamed: 0,name,seed,precision,recall,f1,f1_m,f1_f,Eq accuracy,eq opportunity,pred equality,unpriv_TPR,unpriv_FPR,priv_TPR,priv_FPR
0,0,unimodal_bias_mf-group_mpnet_sent,0,0.929159,0.927273,0.926964,0.877117,0.953358,0.920030,0.900000,1.408046,0.900000,0.172414,1.000000,0.122449
1,1,unimodal_bias_mf-group_mpnet_sent,1,0.927386,0.927273,0.927192,0.877193,0.953522,0.919951,0.881111,1.351724,0.866667,0.137931,0.983607,0.102041
2,2,unimodal_bias_mf-group_mpnet_sent,1123,0.918511,0.915152,0.914619,0.859389,0.943915,0.910452,0.900000,1.448276,0.900000,0.206897,1.000000,0.142857
3,3,unimodal_bias_mf-group_mpnet_sent,3407,0.933333,0.933333,0.933299,0.894672,0.953522,0.938282,0.881111,1.013793,0.866667,0.103448,0.983607,0.102041
4,4,unimodal_bias_mf-group_mpnet_sent,42,0.934632,0.933333,0.933110,0.876737,0.962873,0.910543,0.948889,2.534483,0.933333,0.206897,0.983607,0.081633


In [8]:
df.groupby("name").mean()

,Unnamed: 0,seed,precision,recall,f1,f1_m,f1_f,Eq accuracy,eq opportunity,pred equality,unpriv_TPR,unpriv_FPR,priv_TPR,priv_FPR
name,,,,,,,,,,,,,,
eqodds,8.0,914.6,0.927000,0.925800,0.925600,0.880600,0.950000,0.928000,0.896000,1.488000,NaN,NaN,NaN,NaN
unimodal_bias_mf-group_mpnet_sent,2.0,914.6,0.928604,0.927273,0.927037,0.877022,0.953438,0.919851,0.902222,1.551264,0.893333,0.165517,0.990164,0.110204
unimodal_bias_mf-mixgender_mpnet_sent,7.0,914.6,0.927877,0.927273,0.927146,0.877024,0.953605,0.919834,0.917736,1.841724,0.893333,0.165517,0.973770,0.089796
unimodal_bias_mf-subgroup_mpnet_sent,12.0,914.6,0.930903,0.928485,0.928121,0.884044,0.951452,0.929072,0.912768,1.450421,0.906667,0.165517,0.993443,0.118367
unimodal_bias_mf-synthetic-mixg_mpnet_sent,17.0,914.6,0.926758,0.923636,0.923173,0.876622,0.947800,0.924954,0.945778,1.838506,0.933333,0.206897,0.986885,0.118367
unimodal_bias_mf-synthetic_mpnet_sent,22.0,914.6,0.913075,0.911515,0.911321,0.855818,0.940573,0.909959,0.905399,1.774138,0.873333,0.186207,0.963934,0.106122
unimodal_bias_oversample_mpnet_sent_keyw,27.0,914.6,0.925553,0.924848,0.924707,0.873589,0.951700,0.917800,0.910508,1.810345,0.886667,0.165517,0.973770,0.093878
unimodal_bias_reweighing_mpnet_sent_keyw,32.0,914.6,0.935271,0.934545,0.934451,0.887500,0.959154,0.925271,0.904418,1.588276,0.886667,0.137931,0.980328,0.085714
unimodal_gspec_f_mpnet_sent_keyw,97.0,914.6,0.933818,0.933333,0.933362,0.894581,0.953664,NaN,NaN,NaN,0.963934,0.070936,NaN,NaN


In [9]:
# setup the values of the unimodal gender specific tests
female_gspec = df[df["name"] == "unimodal_gspec_f_mpnet_sent_keyw"]
male_gspec = df[df["name"] == "unimodal_gspec_m_mpnet_sent_keyw"]

for seed in df["seed"].unique():
    female_row = female_gspec[female_gspec["seed"] == seed]
    male_row = male_gspec[male_gspec["seed"] == seed]

    male_tpr, male_fpr, male_f1 = male_row.unpriv_TPR.item(), male_row.unpriv_FPR.item(), male_row.f1.item()
    female_tpr, female_fpr, female_f1 = female_row.unpriv_TPR.item(), female_row.unpriv_FPR.item(), female_row.f1.item()

    # calculate the eqacc, eqopp, and predeq
    eqacc = male_f1 / female_f1
    eqopp = male_tpr / female_tpr
    predeq = male_fpr / female_fpr

    # set the information back into the original setup
    df.loc[(df["name"] == "unimodal_gspec_f_mpnet_sent_keyw") & (df["seed"] == seed), "eq opportunity"] = eqopp
    df.loc[(df["name"] == "unimodal_gspec_f_mpnet_sent_keyw") & (df["seed"] == seed), "pred equality"] = predeq


In [10]:
df.groupby("name").mean()

,Unnamed: 0,seed,precision,recall,f1,f1_m,f1_f,Eq accuracy,eq opportunity,pred equality,unpriv_TPR,unpriv_FPR,priv_TPR,priv_FPR
name,,,,,,,,,,,,,,
eqodds,8.0,914.6,0.927000,0.925800,0.925600,0.880600,0.950000,0.928000,0.896000,1.488000,NaN,NaN,NaN,NaN
unimodal_bias_mf-group_mpnet_sent,2.0,914.6,0.928604,0.927273,0.927037,0.877022,0.953438,0.919851,0.902222,1.551264,0.893333,0.165517,0.990164,0.110204
unimodal_bias_mf-mixgender_mpnet_sent,7.0,914.6,0.927877,0.927273,0.927146,0.877024,0.953605,0.919834,0.917736,1.841724,0.893333,0.165517,0.973770,0.089796
unimodal_bias_mf-subgroup_mpnet_sent,12.0,914.6,0.930903,0.928485,0.928121,0.884044,0.951452,0.929072,0.912768,1.450421,0.906667,0.165517,0.993443,0.118367
unimodal_bias_mf-synthetic-mixg_mpnet_sent,17.0,914.6,0.926758,0.923636,0.923173,0.876622,0.947800,0.924954,0.945778,1.838506,0.933333,0.206897,0.986885,0.118367
unimodal_bias_mf-synthetic_mpnet_sent,22.0,914.6,0.913075,0.911515,0.911321,0.855818,0.940573,0.909959,0.905399,1.774138,0.873333,0.186207,0.963934,0.106122
unimodal_bias_oversample_mpnet_sent_keyw,27.0,914.6,0.925553,0.924848,0.924707,0.873589,0.951700,0.917800,0.910508,1.810345,0.886667,0.165517,0.973770,0.093878
unimodal_bias_reweighing_mpnet_sent_keyw,32.0,914.6,0.935271,0.934545,0.934451,0.887500,0.959154,0.925271,0.904418,1.588276,0.886667,0.137931,0.980328,0.085714
unimodal_gspec_f_mpnet_sent_keyw,97.0,914.6,0.933818,0.933333,0.933362,0.894581,0.953664,NaN,0.899135,1.439080,0.963934,0.070936,NaN,NaN


In [11]:
df.groupby("name").std()

,Unnamed: 0,seed,precision,recall,f1,f1_m,f1_f,Eq accuracy,eq opportunity,pred equality,unpriv_TPR,unpriv_FPR,priv_TPR,priv_FPR
name,,,,,,,,,,,,,,
eqodds,4.743416,1473.777222,0.012000,0.013008,0.013446,0.022898,0.010747,0.021679,0.033615,0.451686,NaN,NaN,NaN,NaN
unimodal_bias_mf-group_mpnet_sent,1.581139,1473.777222,0.006371,0.007423,0.007588,0.012476,0.006703,0.011344,0.027744,0.575875,0.027889,0.044960,0.008979,0.023269
unimodal_bias_mf-mixgender_mpnet_sent,1.581139,1473.777222,0.006893,0.007423,0.007536,0.027966,0.006474,0.033742,0.026517,0.580416,0.014907,0.056661,0.018691,0.011178
unimodal_bias_mf-subgroup_mpnet_sent,1.581139,1473.777222,0.017636,0.019825,0.020207,0.032036,0.016965,0.025972,0.029364,0.224763,0.027889,0.051146,0.014663,0.046538
unimodal_bias_mf-synthetic-mixg_mpnet_sent,1.581139,1473.777222,0.011362,0.013953,0.014312,0.021918,0.014303,0.021682,0.024952,0.479310,0.023570,0.042233,0.007331,0.036507
unimodal_bias_mf-synthetic_mpnet_sent,1.581139,1473.777222,0.012380,0.011814,0.011734,0.018959,0.012464,0.020405,0.051145,0.188909,0.064118,0.039316,0.021374,0.026609
unimodal_bias_oversample_mpnet_sent_keyw,1.581139,1473.777222,0.016594,0.016926,0.017037,0.028856,0.012213,0.022135,0.025336,0.418081,0.029814,0.044960,0.014663,0.030951
unimodal_bias_reweighing_mpnet_sent_keyw,1.581139,1473.777222,0.010977,0.011658,0.011767,0.023824,0.008309,0.022382,0.035553,0.468099,0.038006,0.059726,0.013716,0.017075
unimodal_gspec_f_mpnet_sent_keyw,1.581139,1473.777222,0.004616,0.004285,0.004293,0.012429,0.006595,NaN,0.035149,0.346013,0.007331,0.010263,NaN,NaN


In [14]:
base_model = "unimodal_mpnet_sent_keyw"
bias_models = ['unimodal_bias_oversample_mpnet_sent_keyw', 'unimodal_bias_mf-group_mpnet_sent', 'unimodal_bias_mf-mixgender_mpnet_sent', 'unimodal_bias_mf-subgroup_mpnet_sent',
                'unimodal_bias_mf-synthetic_mpnet_sent', 'unimodal_bias_mf-synthetic-mixg_mpnet_sent', 'unimodal_bias_reweighing_mpnet_sent_keyw', "eqodds", "unimodal_gspec_f_mpnet_sent_keyw"]
columns = ["f1", "f1_m", "f1_f", "Eq accuracy", "eq opportunity", "pred equality"]

# go over each comparison model and compute the t-test
base_info = df[df["name"] == base_model]

# get all the t-test statistics
pstats_info, tstats_info = [], []
for comp_name in bias_models:
    pstats, tstats = [], []
    comp_info = df[df["name"] == comp_name]
    print(f"Comparing {base_model} with {comp_name}")
    
    # for each column perform the paired t-test
    for column in columns:
        model1_scores = base_info[column].values
        model2_scores = comp_info[column].values
        
        t_statistic, p_value = ttest_rel(model1_scores, model2_scores, alternative="two-sided")
        pstats.append(p_value)
        tstats.append(t_statistic)
        
    pstats_info.append((comp_name, *pstats))
    tstats_info.append((comp_name, *tstats))

# make the dataframes
df_ptstats = pd.DataFrame(pstats_info, columns=['name', *columns])
df_ttstats = pd.DataFrame(tstats_info, columns=['name', *columns])

df_ptstats

Comparing unimodal_mpnet_sent_keyw with unimodal_bias_oversample_mpnet_sent_keyw
Comparing unimodal_mpnet_sent_keyw with unimodal_bias_mf-group_mpnet_sent
Comparing unimodal_mpnet_sent_keyw with unimodal_bias_mf-mixgender_mpnet_sent
Comparing unimodal_mpnet_sent_keyw with unimodal_bias_mf-subgroup_mpnet_sent
Comparing unimodal_mpnet_sent_keyw with unimodal_bias_mf-synthetic_mpnet_sent
Comparing unimodal_mpnet_sent_keyw with unimodal_bias_mf-synthetic-mixg_mpnet_sent
Comparing unimodal_mpnet_sent_keyw with unimodal_bias_reweighing_mpnet_sent_keyw
Comparing unimodal_mpnet_sent_keyw with eqodds
Comparing unimodal_mpnet_sent_keyw with unimodal_gspec_f_mpnet_sent_keyw


,name,f1,f1_m,f1_f,Eq accuracy,eq opportunity,pred equality
0,unimodal_bias_oversample_mpnet_sent_keyw,0.642809,0.408715,0.757998,0.320716,0.513445,0.112155
1,unimodal_bias_mf-group_mpnet_sent,0.869067,0.427329,0.641437,0.216443,0.725241,0.517774
2,unimodal_bias_mf-mixgender_mpnet_sent,0.742757,0.377870,0.587845,0.400681,0.006968,0.242857
3,unimodal_bias_mf-subgroup_mpnet_sent,0.974011,0.817775,0.856090,0.708904,0.447953,0.616341
4,unimodal_bias_mf-synthetic_mpnet_sent,0.070468,0.067356,0.140766,0.093027,0.568631,0.057731
5,unimodal_bias_mf-synthetic-mixg_mpnet_sent,0.157157,0.200379,0.722802,0.452426,0.059158,0.142469
6,unimodal_bias_reweighing_mpnet_sent_keyw,0.439911,0.999404,0.091018,0.544897,0.727785,0.513096
7,eqodds,0.789445,0.709026,0.984340,0.651755,0.995844,0.216911
8,unimodal_gspec_f_mpnet_sent_keyw,0.373901,0.373901,0.373901,NaN,0.373901,0.183849


In [13]:
df_ttstats

,name,f1,f1_m,f1_f,Eq accuracy,eq opportunity,pred equality
0,unimodal_bias_oversample_mpnet_sent_keyw,0.500809,0.921983,-0.329944,1.132482,-0.716220,-2.030449
1,unimodal_bias_mf-group_mpnet_sent,0.175698,0.882546,-0.502940,1.466332,-0.377117,-0.708441
2,unimodal_bias_mf-mixgender_mpnet_sent,0.351774,0.990797,-0.588463,0.939468,-5.102947,-1.368933
3,unimodal_bias_mf-subgroup_mpnet_sent,0.034660,0.246021,-0.193372,0.401020,-0.840462,-0.542412
4,unimodal_bias_mf-synthetic_mpnet_sent,2.449707,2.491784,1.832818,2.196388,-0.620337,-2.637501
5,unimodal_bias_mf-synthetic-mixg_mpnet_sent,1.738260,1.531601,0.380668,0.831541,-2.614208,-1.822464
6,unimodal_bias_reweighing_mpnet_sent_keyw,-0.856681,0.000794,-2.215986,0.660733,-0.373418,-0.716850
7,eqodds,0.285486,0.400840,-0.020881,0.486978,0.005541,-1.464506
8,unimodal_gspec_f_mpnet_sent_keyw,-1.000000,-1.000000,-1.000000,NaN,-1.000000,-1.604608
